In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import getpass

from datetime import datetime

import lxml



# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# %pip install transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
# %pip install xlrd

# This one allows me not to put plt.show() in every graph.
%matplotlib inline


c:\Users\andre\anaconda3\envs\DataScience\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
user = getpass.getuser()
rel_path = '/Texas Exchange/Project'


in_path = '/Users/' + user + rel_path + '/input/'
out_path = '/Users/' + user + rel_path + '/output/'


In [25]:
excel_data = "Pro 0-3712 FINAL ORDER.xls"

df = pd.read_excel(in_path+ excel_data)

df1 = df.copy()


df1=df[['Title', 'StoreId', 'entryDate', 'pubdate', 'pubtitle']]

df1


,Title,StoreId,entryDate,pubdate,pubtitle
0,"Apple Inc. Stock Falls Monday, Underperforms M...",2877616354,"Oct 16, 2023","Oct 16, 2023",Dow Jones Institutional News
1,Zacks Investment Research: Apple (AAPL) Stock ...,2877513694,"Oct 16, 2023","Oct 16, 2023",Newstex Finance & Accounting Blogs
2,Apple (AAPL) Stock Sinks As Market Gains: What...,2877508989,"Oct 16, 2023","Oct 16, 2023",Zacks Investment Research [BLOG]
3,Thinking about trading options or stock in App...,2877344743,"Oct 16, 2023","Oct 16, 2023",PR Newswire
4,American Banking and Market News - American Co...,2877300797,"Oct 16, 2023","Oct 16, 2023",Newstex Finance & Accounting Blogs
...,...,...,...,...,...
11129,Apple Inc Files Patent Application for Portabl...,2759671243,"Dec 31, 2022","Dec 31, 2022",Indian Patents News
11130,Apple Inc Files Patent Application for Improve...,2759696836,"Dec 31, 2022","Dec 31, 2022",Indian Patents News
11131,Here's what's in store for Apple's stock in 2023,2759717052,"Dec 29, 2022","Dec 29, 2022",CE Noticias Financieras
11132,"Bulls And Bears Of The Week: Apple, AMC, Meta,...",2759717080,"Dec 31, 2022","Dec 31, 2022",Benzinga Newswires


In [26]:
df1.dtypes
df1['proper_date'] = pd.to_datetime(df1['pubdate'], dayfirst=True)

    
df2 = df1.copy()


In [27]:
df2

,Title,StoreId,entryDate,pubdate,pubtitle,proper_date
0,"Apple Inc. Stock Falls Monday, Underperforms M...",2877616354,"Oct 16, 2023","Oct 16, 2023",Dow Jones Institutional News,2023-10-16
1,Zacks Investment Research: Apple (AAPL) Stock ...,2877513694,"Oct 16, 2023","Oct 16, 2023",Newstex Finance & Accounting Blogs,2023-10-16
2,Apple (AAPL) Stock Sinks As Market Gains: What...,2877508989,"Oct 16, 2023","Oct 16, 2023",Zacks Investment Research [BLOG],2023-10-16
3,Thinking about trading options or stock in App...,2877344743,"Oct 16, 2023","Oct 16, 2023",PR Newswire,2023-10-16
4,American Banking and Market News - American Co...,2877300797,"Oct 16, 2023","Oct 16, 2023",Newstex Finance & Accounting Blogs,2023-10-16
...,...,...,...,...,...,...
11129,Apple Inc Files Patent Application for Portabl...,2759671243,"Dec 31, 2022","Dec 31, 2022",Indian Patents News,2022-12-31
11130,Apple Inc Files Patent Application for Improve...,2759696836,"Dec 31, 2022","Dec 31, 2022",Indian Patents News,2022-12-31
11131,Here's what's in store for Apple's stock in 2023,2759717052,"Dec 29, 2022","Dec 29, 2022",CE Noticias Financieras,2022-12-29
11132,"Bulls And Bears Of The Week: Apple, AMC, Meta,...",2759717080,"Dec 31, 2022","Dec 31, 2022",Benzinga Newswires,2022-12-31


In [28]:
df2['Title']

0        Apple Inc. Stock Falls Monday, Underperforms M...
1        Zacks Investment Research: Apple (AAPL) Stock ...
2        Apple (AAPL) Stock Sinks As Market Gains: What...
3        Thinking about trading options or stock in App...
4        American Banking and Market News - American Co...
                               ...                        
11129    Apple Inc Files Patent Application for Portabl...
11130    Apple Inc Files Patent Application for Improve...
11131     Here's what's in store for Apple's stock in 2023
11132    Bulls And Bears Of The Week: Apple, AMC, Meta,...
11133    Tesla, Apple, Southwest Airlines: Stocks That ...
Name: Title, Length: 11134, dtype: object

In [5]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")



In [13]:
def sentiment_score(text):
    tokens=tokenizer.encode(text, return_tensors='pt')
    result =model(tokens)
    return int((result.logits))-1

In [14]:
sentiment_score("This is very good news")

AttributeError: module 'torch' has no attribute 'arg'

In [12]:
sentiment_score("This is good news")

SequenceClassifierOutput(loss=None, logits=tensor([[-2.9078, -1.2164,  4.6350]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [31]:
# for index, row in df2.iterrows():
#     df2['Sentiment']=sentiment_score(df2.loc[index, 'extracted_texts'])

# sentiment_score(df2.loc[1, 'extracted_texts'])

df2['sentiment']= df2['Title'].apply(lambda x: sentiment_score(x[:1000]))


In [32]:
df2['sentiment'].describe()

count    11134.000000
mean         0.025507
std          0.501749
min         -1.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: sentiment, dtype: float64

In [33]:
df3 = df2.groupby(['proper_date','pubtitle']).agg({'sentiment':'mean'}).reset_index()

In [35]:
df2.to_excel(out_path + 'df_with_sentiment_all_11000.xlsx', index=False)


In [36]:
df3.to_excel(out_path + 'df_with_sentiment_grouped_11000.xlsx', index=False)